# 수강신청 결과 크롤러

* 마일리지 결과 페이지의 문제점 : 마일리지 커트라인이 나와있지 않음
* 매번 과목 마다 돌아다니며 마일리지 컷을 확인하는 것도 매우매우 귀찮음
* 크롤링으로 해결!

In [5]:
!pip install selenium
!pip install bs4
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import html5lib

path = "C:\\Users\WcrossW\chromedriver.exe"
driver = webdriver.Chrome(path)
# 크롬 웹 드라이버 객체 생성.
# 크롬드라이버로 크롬을 조작.

driver.get("https://portal.yonsei.ac.kr/")
#드라이버 객체에서 원하는 url로 이동.

###############################################################
######################## 기본 동작  ###########################
driver.refresh() # 새로 고침
# driver.quit() # 브라우저 종료

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_7396/2788524185.py, line 10)

In [10]:
subjects = [x for x in input('공백을 구분자로 관심있는 수업명을 알려주세요 : ').split()]

공백을 구분자로 관심있는 수업명을 알려주세요 : 컴퓨터비전 이론통계학 데이터마이닝


In [11]:
subjects

['컴퓨터비전', '이론통계학', '데이터마이닝']

In [12]:
table_df_list=[]

for i, subject in enumerate(subjects) : 
    
    # 소스코드에서 컨트롤하고 싶은 부분을 찾아서 우클릭 -> 'copy selector'
    # 'find.element_by_css_selector'를 통해 copy 해온 것의 element를 찾는다.
    # element를 찾아 클릭, 텍스트 뽑기, 텍스트 입력 등 다양한 조작이 가능하다.

    elem = driver.find_element_by_css_selector('#win_into_5_sugang')
    # 위 코드로 수강편람창 선택
    elem.click()
    # 위 코드로 선택한 것을 클릭

    # 수강편람 창이 새 탭으로 띄어진다. 
    # 현재 활성 창은 포털 창이기 때문에, 활성 창을 수강편람 창으로 변경해 주어야 한다.
    #myForm > table > tbody > tr:nth-child(2) > td:nth-child(2) > select > option:nth-child(2)

    driver.switch_to_window(driver.window_handles[1]) 
    driver.get_window_position(driver.window_handles[1])

    # driver.window_handles[0]은 본래의 창을 의미한다.
    # driver.window_handles[1]은 새로 띄어 진 창을 의미한다.

    # 교과목 명을 검색 
    elem = driver.find_element_by_css_selector('#myForm > table > tbody > tr:nth-child(2) > td:nth-child(2) > select > option:nth-child(2)')
    elem.click()
    
    
    # 검색창에 'subject' 입력
    elem = driver.find_element_by_css_selector('#myForm > table > tbody > tr:nth-child(2) > td:nth-child(2) > input[type=text]:nth-child(3)')
    elem.send_keys(subject)
    elem.send_keys(Keys.RETURN)   # 엔터를 쳐서 입력
    elem.send_keys
    
    time.sleep(5)

    # 검색 결과 가장 최상단의 결과를 불러옴
    elem = driver.find_element_by_css_selector('#row0jqxgrid > div:nth-child(7) > span') 
    # 최상단의 네가지 버튼의 리스트를 갖고옴 : 동일 교과목 조회, 수업계획서 조회 등 
    elem = elem.find_elements_by_tag_name("a")
    # 그 중 4번째 버튼인, 마일리지 결과 조회 버튼을 갖고옴 
    elem = elem[3]
    # 클릭 
    elem.click()

    # 마일리지 결과 조회 창으로 전환 
    driver.switch_to_window(driver.window_handles[2])
    driver.get_window_position(driver.window_handles[2])
    
    time.sleep(3)

    #  먼저 selenium을  통해 해당 페이지의 html소스를 가져옵니다. 
    # 이 html 소스코드를 BeautifulSoup에 넣어주면 태그명, css selector 등으로 html 파싱이 가능합니다.

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # parsing이란? 

    time.sleep(3)

    # html 소스코드에서 (table 태그 모두 수집)
    tables = soup.find_all('table')

    table = tables[1]  # 두번째 표가 우리가 긁어오고 싶은 표
    table2 = tables[2]
    # 테이블 html 정보를 문자열로 변경하기
    table_html = str(table)
    table_html2 = str(table2)
    # 판다스의 read_html 로 테이블 정보 읽기
    
    target_table2 = pd.read_html(table_html2)
    target_table2 = pd.DataFrame(target_table2[0])
    
    target_table2 = target_table2.rename(columns=target_table2.iloc[0])
    target_table2 = target_table2.drop(target_table2.index[0])
    target_table2 = target_table2.drop(target_table2.index[0])
    target_table2 = target_table2[['순위','마일리지','수강여부']]
    target_table2_sugang = target_table2[target_table2["수강여부"] == 'O']
    minmile = target_table2_sugang[['마일리지']].min().min()    
    minmile = pd.DataFrame({'최소마일리지':[0,0,0,minmile]})
    minmile = minmile.drop(minmile.index[0])
    minmile = minmile.drop(minmile.index[0])
    minmile = minmile.drop(minmile.index[0])

    
    # target_table은 리스트 형태이다. 
    # 리스트 안에 데이터프레임이 들어있기 때문에, 인덱싱을 이용해서 데이터 프레임을 불러온다. 
    target_table = pd.read_html(table_html)
    table_df_list.append(target_table[0].loc[(0,3),:])
    table_df_list.append(minmile)
    table2 = tables[2]
    
      # 테이블 html 정보를 문자열로 변경하기
    table_html2 = str(table2)
    # 판다스의 read_html 로 테이블 정보 읽기
    
    # target_table은 리스트 형태이다. 
    # 리스트 안에 데이터프레임이 들어있기 때문에, 인덱싱을 이용해서 데이터 프레임을 불러온다. 
    
    # 켜졌던 창들을 다 꺼준다. 
    driver.switch_to_window(driver.window_handles[2])
    driver.close()
    driver.switch_to_window(driver.window_handles[1])
    driver.close()
    driver.switch_to_window(driver.window_handles[0])

<ipython-input-12-50fa623e830e>:18: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])
C:\Users\cross\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")
<ipython-input-12-50fa623e830e>:47: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[2])
<ipython-input-12-50fa623e830e>:101: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[2])
<ipython-input-12-50fa623e830e>:103: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])
<ipython-input-12-50fa623e830e>:105: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0])


In [13]:
pd.concat(table_df_list)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,최소마일리지
0,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자정원(2전공포함),학년별정원,학년별정원,학년별정원,학년별정원,교환학생가능여부,MaxMileage,마일리지,마일리지,마일리지,NaN
3,CSI4116-01-00,비대면-컴퓨터비전,3,김선주,"화3,4,목6",실시간온라인,100,156,90 (N),0,0,0,0,O,36,1,36,27.12,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
0,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자정원(2전공포함),학년별정원,학년별정원,학년별정원,학년별정원,교환학생가능여부,MaxMileage,마일리지,마일리지,마일리지,NaN
3,STA4115-01-00,비대면-이론통계학(1),3,강승호,"월7,8/수8",동영상콘텐츠/실시간온라인,85,25,60 (Y),0,0,0,0,X,12,1,12,6.52,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
0,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자정원(2전공포함),학년별정원,학년별정원,학년별정원,학년별정원,교환학생가능여부,MaxMileage,마일리지,마일리지,마일리지,NaN
3,STA4103-01-00,비대면-데이터마이닝,3,김현중,"월5,6/수11",실시간온라인/동영상콘텐츠,51,70,50 (Y),0,0,0,0,X,12,1,12,9.44,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
